<a href="https://colab.research.google.com/github/FR-Schwartz/IDS705_Team10/blob/main/10_code/auto_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import print_function
import keras
from keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Input

In [3]:
from google.colab import drive
import os
import pickle
import tensorflow_gcs_config
import shutil
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import nibabel as nib
import tensorflow_hub as hub
drive.mount('/content/MyDrive/')
os.chdir('/content/MyDrive/MyDrive/IDS705_Final') #change to file path on your disk

Mounted at /content/MyDrive/


In [4]:
#Create Train / Val / Test Split
subfolders = os.listdir("/content/MyDrive/MyDrive/IDS705_Final/Data/Train")
np.random.seed(101)
split = np.random.choice(["Train","Val","Test"], len(subfolders), p=[0.6, 0.2, 0.2])
train_ids = [subfolders[i] for i,v in enumerate(split) if v=="Train"]
val_ids = [subfolders[i] for i,v in enumerate(split) if v=="Val"]
test_ids = [subfolders[i] for i,v in enumerate(split) if v=="Test"]

In [45]:
#Helper Functions
def parse_tfrecord(example):
  """
  This function helps in parsing tfrecord files when creating a TF Dataset object
  """
  feature = {'image': tf.io.FixedLenFeature([240, 240, 155, 4], tf.float32),
             'label': tf.io.FixedLenFeature([240, 240, 155], tf.int64)}
  parsed_example = tf.io.parse_single_example(example, feature)
  return parsed_example

def get_image_and_label(features):
  """
  Extract Image and Label Object from tfrecord files
  """
  image, label = features['image'], features['label']
  return image, label

def get_image_and_label_auto(features):
  """
  Extract Image and Label Object from tfrecord files
  """
  image = features['image']
  return image, image

def get_dataset(tfrecord_names):
  """
  Create TF dataset files that can be fed into model functions
  """
  dataset = (tf.data.TFRecordDataset(tfrecord_names)
             .map(parse_tfrecord)
             .map(get_image_and_label_auto))

  return dataset

In [46]:
#Create dataset objects
train_dataset = get_dataset([f'/content/MyDrive/MyDrive/IDS705_Final/Data/train_tf/{sf}.tfrecord' for sf in train_ids])
test_dataset = get_dataset([f'/content/MyDrive/MyDrive/IDS705_Final/Data/train_tf/{sf}.tfrecord' for sf in test_ids])
val_dataset = get_dataset([f'/content/MyDrive/MyDrive/IDS705_Final/Data/train_tf/{sf}.tfrecord' for sf in val_ids])
mini_dataset = get_dataset([f'/content/MyDrive/MyDrive/IDS705_Final/Data/train_tf/{sf}.tfrecord' for sf in subfolders[100:228]])
minival_dataset = get_dataset([f'/content/MyDrive/MyDrive/IDS705_Final/Data/train_tf/{sf}.tfrecord' for sf in subfolders[500:508]])

In [47]:
mini_dataset

<MapDataset element_spec=(TensorSpec(shape=(240, 240, 155, 4), dtype=tf.float32, name=None), TensorSpec(shape=(240, 240, 155, 4), dtype=tf.float32, name=None))>

In [264]:
def encoder(input_layer):
  # Elements in layer represent in this order batchSize, height, width, channels
  print("*****Encoder*****", input_layer)

  x1 = tf.keras.layers.BatchNormalization()(input_layer)
  x1 = tf.keras.layers.Conv3D( 4, (1,1,1), strides=(1,1,1), activation='relu')(x1)

  print("x1", x1.shape)

  x2 = tf.keras.layers.Conv3D( 8, 3, strides=(2,2,2), activation='relu', padding='same')(x1)
  x2 = tf.keras.layers.Conv3D( 8, 3, strides=(1,1,1), activation='relu')(x2)

  print("x2", x2.shape)

  x3 = tf.keras.layers.Conv3D( 16, 3, strides=(2,2,2), activation='relu', padding='same')(x2)

  print("x3", x3.shape)

  x4 = tf.keras.layers.Conv3D( 32, 3, strides=(2,2,2), activation='relu', padding='same')(x3)
  x4 = tf.keras.layers.Conv3D( 32, 3, strides=(1,1,1), activation='relu')(x4)

  print("x4", x4.shape)

  x5 = tf.keras.layers.Conv3D( 64, 3, strides=(2,2,2), activation='relu', padding='same')(x4)
  x5 = tf.keras.layers.Conv3D( 64, 3, strides=(1,1,1), activation='relu')(x5)

  print("x5", x5.shape)

  x6 = tf.keras.layers.Conv3D( 128, 3, strides=(2,2,2), activation='relu', padding='same')(x5)
  x6 = tf.keras.layers.Conv3D( 128, 3, strides=(1,1,1), activation='relu')(x6)

  print("x6", x6.shape)

  return x6


In [631]:
def decoder(encoded_layer):
  print("*****Decoder*****", encoded_layer)

  x1 = tf.keras.layers.Conv3DTranspose( 64, 4, strides=(1,1,1), activation='relu')(encoded_layer)
  x1 = tf.keras.layers.Conv3DTranspose( 64, 2, strides=(2,2,2), activation='relu', padding='valid')(x1)


  print("x1", x1.shape)

  
  x3 = tf.keras.layers.Conv3DTranspose( 32, 3, strides=(2,2,3), activation='relu', padding="valid")(x1)
  x3 = tf.keras.layers.Conv3DTranspose( 32, 1, strides=(1,1,2), activation='relu')(x3)

  print("x3", x3.shape)

  x4 = tf.keras.layers.Conv3DTranspose( 16, 2, strides=(2,2,1), activation='relu', padding="valid")(x3)
  x4 = tf.keras.layers.Conv3DTranspose( 16, 2, strides=(1,1,1), activation='relu')(x4)

  print("x4", x4.shape)

  x5 = tf.keras.layers.Conv3DTranspose( 8, 3, strides=(2,2,2), activation='relu', padding="valid")(x4)
  x5 = tf.keras.layers.Conv3DTranspose( 8, 2, strides=(1,1,1), activation='relu')(x5)

  print("x5", x5.shape)

  x6 = tf.keras.layers.Conv3DTranspose( 4, 2, strides=(2,2,1), activation='relu', padding="valid")(x5)
  x6 = tf.keras.layers.Conv3DTranspose( 4, (1,1,29), strides=(1,1,1), activation='relu', padding="valid")(x6)
  


  print("x6", x6.shape)

  return x6
  


In [632]:
# Combine Encoder and Deocder layers
from keras.models import Model, Input

input_layer = tf.keras.layers.Input(shape=(240,240,155,4))
e_l = encoder(input_layer)
output = decoder(e_l)


autoencoder = Model(inputs = input_layer, outputs = output)

*****Encoder***** KerasTensor(type_spec=TensorSpec(shape=(None, 240, 240, 155, 4), dtype=tf.float32, name='input_288'), name='input_288', description="created by layer 'input_288'")
x1 (None, 240, 240, 155, 4)
x2 (None, 118, 118, 76, 8)
x3 (None, 59, 59, 38, 16)
x4 (None, 28, 28, 17, 32)
x5 (None, 12, 12, 7, 64)
x6 (None, 4, 4, 2, 128)
*****Decoder***** KerasTensor(type_spec=TensorSpec(shape=(None, 4, 4, 2, 128), dtype=tf.float32, name=None), name='conv3d_3034/Relu:0', description="created by layer 'conv3d_3034'")
x1 (None, 14, 14, 10, 64)
x3 (None, 29, 29, 60, 32)
x4 (None, 59, 59, 62, 16)
x5 (None, 120, 120, 126, 8)
x6 (None, 240, 240, 155, 4)


In [633]:
autoencoder.compile(optimizer="adadelta", loss="KLDivergence")
autoencoder.summary()

Model: "model_265"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_288 (InputLayer)      [(None, 240, 240, 155, 4  0         
                             )]                                  
                                                                 
 batch_normalization_287 (Ba  (None, 240, 240, 155, 4)  16       
 tchNormalization)                                               
                                                                 
 conv3d_3025 (Conv3D)        (None, 240, 240, 155, 4)  20        
                                                                 
 conv3d_3026 (Conv3D)        (None, 120, 120, 78, 8)   872       
                                                                 
 conv3d_3027 (Conv3D)        (None, 118, 118, 76, 8)   1736      
                                                                 
 conv3d_3028 (Conv3D)        (None, 59, 59, 38, 16)    34

In [634]:
batchsize = 4
shufflesize = 8
mini_dataset_trainable = mini_dataset.shuffle(shufflesize).batch(batchsize)
minival_dataset_validable = minival_dataset.batch(batchsize)
train_dataset_trainable = train_dataset.shuffle(shufflesize).batch(batchsize)
val_dataset_validable = val_dataset.batch(batchsize)

In [ ]:
autoencoder.fit(train_dataset_trainable, epochs = 2, batch_size = 4, shuffle = False,validation_data=val_dataset_validable)

In [ ]:
# #DICE Coefficent definition used as a metric in compilation
# import keras.backend as k
# def dice_coef(y_pred, y_true):
#     inter = k.sum(y_true[:,:,-1]*y_pred[:,:,-1])
#     return (2*inter + 1)/(k.sum(y_true[:,:,-1]) + k.sum(y_pred[:,:,-1]) + 1)